In [17]:
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [19]:
directory = "/Users/keon/fastcampus/data-engineering/01-spark/data"
filename = "fhvhv_tripdata_2020-03.csv"

In [20]:
data = spark.read.csv(f"file:///{directory}/{filename}", inferSchema = True, header = True)

In [21]:
data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [22]:
data.createOrReplaceTempView("mobility_data")

In [23]:
spark.sql("select * from mobility_data limit 5").show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [24]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-03|697880|
| 2020-03-04|707879|
| 2020-03-05|731165|
| 2020-03-06|872012|
| 2020-03-07|886071|
| 2020-03-08|731222|
| 2020-03-10|626474|
| 2020-03-09|628940|
| 2020-03-11|628601|
| 2020-03-12|643257|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-16|391518|
| 2020-03-17|312298|
| 2020-03-18|269232|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [25]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#313]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#312]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#312], [pickup_date#312, count(1) AS trips#313L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(pickup_datetime#204,  , -1)[0] AS pickup_date#312]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#202,dispatching_base_num#203,pickup_datetime#204,dropoff_datetime#205,PULocationID#206,DOLocationID#207,SR_Flag#208])
            +- Relation [hvfhs_license_num#202,dispatching_base_num#203,pickup_datetime#204,dropoff_datetime#205,PULocationID#206,DOLocationID#207,SR_Flag#208] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#312], [pickup_date#312, count(1) AS trips#313L]
+- Proj